In [8]:
%matplotlib nbagg
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
#########################################################################
### This file is used to generate Figure 4 in the paper
###
### Tight bounds on the mutual coherence of sensing matrices
### for Wigner D-functions on regular grid - Arya Bangun, Arash Behboodi
### and Rudolf Mathar 
###
### Created by Arya Bangun
#########################################################################
import matplotlib.pyplot as plt
import numpy as np
from grad_algorithms import GradientAlgorithms
from metric import BoundCoherence
from matrix import Matrix

types = ['snf','partial']
if types[0] == 'SH':
    B = 10
    N = B**2
elif types[0] == 'Wigner':

    B = 4 # Bandwidth of the spherical harmonics
    N = B*(2*B-1)*(2*B+1)//3 # Dimension of spherical harmonics sensing matrix
else:
    B = 8 
    N = 2*B**2 + 4*B
m = np.arange(17,N,8).astype(np.int64) # Samples

 

## Preallocation
graddes = np.zeros(len(m), dtype = np.float64);
adam = np.copy(graddes)
adadelta = np.copy(graddes)
adagrad = np.copy(graddes)
legbound = np.copy(graddes)
welchbound = np.copy(graddes)
eps = 1e-4
max_iter = 20

for ii in range(len(m)):
    
    ## Generate samples
    angles = np.zeros((m[ii],3), dtype = np.float64)
    if types[1] == 'all':
        
        angles[:,0] = np.random.rand(m[ii])*np.pi
        angles[:,1] = np.random.rand(m[ii])*2*np.pi
        angles[:,2] = np.random.rand(m[ii])*2*np.pi
    else:
        angles[:,0] = np.arccos(np.linspace(-1,1,m[ii]))
        angles[:,1] = np.random.rand(m[ii])*2*np.pi
        angles[:,2] = np.random.rand(m[ii])*2*np.pi
    
    ## Generate Matrix
    matrix = Matrix(B = B,
                    types = types,
                    angles = angles)
    ## Gradient
    gr_algo = GradientAlgorithms(matrix, eps, max_iter)
    
    ## Generate bound
    Bound = BoundCoherence(m[ii],N,B) # Generate the Bound in the main theorem
    
    ## Calculate gradient
    ang_all_graddes = gr_algo.graddes_coh # use simple gradient descent
    print("GrDe     -- Samples %s, Coherence  %s,  Welch Bound %s, and Legendre Bound %s" %(m[ii],ang_all_graddes,Bound[0], Bound[1])) 
    ang_all_adam = gr_algo.adam_coh # use adam 
    print("ADAM     -- Samples %s, Coherence  %s,  Welch Bound %s, and Legendre Bound %s" %(m[ii],ang_all_adam,Bound[0], Bound[1]))
    ang_all_adadelta = gr_algo.adadelta_coh # use adadelta
    print("AdaDelta -- Samples %s, Coherence  %s,  Welch Bound %s, and Legendre Bound %s" %(m[ii],ang_all_adadelta,Bound[0], Bound[1]))   
    ang_all_adagrad = gr_algo.adagrad_coh # use adagrad
    print("AdaGrad  -- Samples %s, Coherence  %s,  Welch Bound %s, and Legendre Bound %s" %(m[ii],ang_all_adagrad,Bound[0], Bound[1]))
                    

    ### File
    graddes[ii] = ang_all_graddes
    adam[ii] = ang_all_adam
    adadelta[ii] = ang_all_adadelta
    adagrad[ii] = ang_all_adagrad
    legbound[ii] = Bound[1]
    welchbound[ii] = Bound[0]



GrDe     -- Samples 17, Coherence  0.7841409436144405,  Welch Bound 0.23000908795119507, and Legendre Bound 0.5894621811533247
ADAM     -- Samples 17, Coherence  0.7841409436144405,  Welch Bound 0.23000908795119507, and Legendre Bound 0.5894621811533247
AdaDelta -- Samples 17, Coherence  0.7841409436144405,  Welch Bound 0.23000908795119507, and Legendre Bound 0.5894621811533247
AdaGrad  -- Samples 17, Coherence  0.7841409436144405,  Welch Bound 0.23000908795119507, and Legendre Bound 0.5894621811533247
GrDe     -- Samples 25, Coherence  0.7802880177555755,  Welch Bound 0.18428853505018536, and Legendre Bound 0.45314919838138296
ADAM     -- Samples 25, Coherence  0.7802880177555755,  Welch Bound 0.18428853505018536, and Legendre Bound 0.45314919838138296
AdaDelta -- Samples 25, Coherence  0.7802880177555755,  Welch Bound 0.18428853505018536, and Legendre Bound 0.45314919838138296
AdaGrad  -- Samples 25, Coherence  0.7802880177555755,  Welch Bound 0.18428853505018536, and Legendre Bound 

KeyboardInterrupt: 

In [ ]:


plt.figure(1)
plt.plot(m,graddes, color = 'r', marker = 'o', linestyle = '--',linewidth = 2, label = 'Gradient Descent in Algorithm ')
#plt.hold(True)
plt.grid(True)
plt.plot(m,adadelta, color = 'k', marker = 'x', linewidth = 2, linestyle = '--', label = 'Ada Delta')
plt.plot(m,adagrad, color = 'c', marker = 's', linestyle = '--', linewidth = 2, label = 'Ada Grad')
plt.plot(m,adam, color = 'g', marker = '*', linewidth = 2, linestyle = '--', label = 'Adam')
plt.plot(m,legbound, color = 'b', marker = 's', linewidth = 2, linestyle = '--', label = 'Bound in Theorem 1')
plt.plot(m,welchbound, color = 'tab:brown', marker = '^', linewidth = 2, linestyle = '--', label = 'Welchbound ')
plt.xlim(m[0],m[len(m)-1])
plt.ylim(0,1)
plt.legend()
plt.xlabel('Samples (m)')
plt.ylabel('Coherence')
plt.show()



In [6]:
matrix.A.shape

(33, 160)

In [7]:
N

160